In [13]:
# 24 Jan, 2022
# Some helper functions

import numpy as np
from scipy import optimize
from math import log, sqrt, ceil, exp
from scipy.stats import binom

"""
KL Inequalities
"""

def KL(Q, P):
    """
    Compute Kullback-Leibler (KL) divergence between distributions Q and P.
    """
    return sum([ q*log(q/p) if q > 0. else 0. for q,p in zip(Q,P) ])
    
def KL_binomial(q, p):
    """
    Compute the KL-divergence between two Bernoulli distributions of probability
    of success q and p. That is, Q=(q,1-q), P=(p,1-p).
    """
    return KL([q, 1.-q], [p, 1.-p])
    
def solve_kl_sup(q, right_hand_side):
    """
    find x such that:
        kl( q || x ) = right_hand_side
        x > q
    """
    f = lambda x: KL_binomial(q, x) - right_hand_side

    if f(1.0-1e-9) <= 0.0:
        return 1.0-1e-9
    else:
        return optimize.brentq(f, q, 1.0-1e-9)

def solve_kl_inf(q, right_hand_side):
    """
    find x such that:
        kl( q || x ) = right_hand_side
        x < q
    """
    f = lambda x: KL_binomial(q, x) - right_hand_side

    if f(1e-9) <= 0.0:
        return 1e-9
    else:
        return optimize.brentq(f, 1e-9, q)

"""
Inverse of Chernoff-Hoeffding's Bound
"""
def solve_CH_left(hatp, right_hand_side):
    """
    find p such that:
        eps = p - hatp >= 0
        kl( p-eps || p) = right_hand_side
        eps < p
    """
    f = lambda x: KL_binomial(hatp, x) - right_hand_side

    if f(1-1e-9) >= 0.0:
        return 1-1e-9
    else:
        return optimize.brentq(f, 1e-9, 1)

def solve_CH_right(p, right_hand_side):
    """
    find eps such that:
        kl( p+eps || p) = right_hand_side
        eps < 1-p
    """
    f = lambda x: KL_binomial(p+x, p) - right_hand_side

    if f(1-p-1e-9) >= 0.0:
        return 1-p-1e-9
    else:
        return optimize.brentq(f, 1e-9, 1-p)

"""
Binomial Tail
"""
    
def solve_BinL(n, k, delta):
    """
    find p such that:
        binom.cdf(k, n, p) > delta
        k/n < p <= 1
    """
    f = lambda x: binom.cdf(k, n, x) - delta
    
    if f(1.0) >= 0.0:
        return 1.0
    else:
        return optimize.brentq(f, k/n, 1.0)
        
def solve_BinR(n, k, delta):
    """
    find p such that:
        1 - binom.cdf(k-1, n, p) > delta
        0 <= p < k/n
    """
    f = lambda x: 1-binom.cdf(k-1, n, x) - delta
    
    if k ==0:
        return 0.0
    elif f(0.0) >= 0.0:
        return 0.0
    else:
        return optimize.brentq(f, 0., k/n)

"""
Bennett's Inequalities
"""

def OBennett(q, Var, n, delta):
    """
    Bennett's inequality with oracle variance Var
    """
    return min(1-1e-9, q + sqrt(2*Var*log(1/delta)/n) + 2*log(1/delta)/(3*n))


def Emp_Bernstein(q, hVar, a, b, n, delta):
    """
    Empirical Bennett's inequality (with empirical variance hVar)
    generalize to Z\in[a,b]
    """
    return min(b, q + sqrt(2*hVar*log(2/delta)/n) + 7*(b-a)*log(2/delta)/(3*n))

def Unexp_Bernstein(q, x2, b, n, delta):
    """
    Unexpected Bernstein's inequality
    """
    gamGridSize = ceil(log(0.5*sqrt(n/log(1/delta)))/log(2))
    bounds = np.zeros(gamGridSize)
    for i in range(gamGridSize):
        gam = 1/(b*2**(i+1)) 
        bounds[i] = gam*x2 + (log(gamGridSize/delta))/(gam*n)
    return min(b, min(bounds))

"""
Empirical Mean and Variance
"""

def Emp_MaV(vec):
    """
    Compute the empirical mean and the empirical variance
    of a sequence of random variables vec
    """
    mean = np.mean(vec)
    var = np.sum((vec-mean)**2)/len(vec-1)
    x2 = np.mean(vec**2)
    return mean, var, x2
    
"""
kl shifted bound
"""
def splitkl(mu, a, b, barSzp, barSzm, right_hand_side):
    """
    Compute the kl shifted bound.
    mu = shift
    Z=(z1,z2,z3), the support of the ternary random variable
    barSzp, barSzm = \hat{Z}_n^+, \hat{Z}_n^-
    """
    barSzpT = 0 if mu==b else barSzp/(b-mu)
    barSzmT = 0 if mu==a else barSzm/(mu-a)
    EzpSUP = solve_kl_sup(barSzpT, right_hand_side)
    EzmINF = solve_kl_inf(barSzmT, right_hand_side)
    return mu + (b-mu)*EzpSUP - (mu-a)*EzmINF

"""
Catoni's bound
"""
def Catoni(q, n, delta):
    CGridSize = ceil(log(0.5*sqrt(n/log(1/delta)))/log(2))
    eps = log(CGridSize/delta)/n
    bounds = np.zeros(CGridSize)
    for i in range(CGridSize):
        C = 1/2**i
        bounds[i] = (1-exp(-(C*q+eps))) / (1-exp(-C))
    return min(bounds)

In [14]:
# 22 Nov, 2021
# Split kl for ternary random variables
# Consider Z\in\{z1,z2,z3\}, z_i\in\R
#
# Usage: python split-kl.py [RATE] [n]
#        RATE        : \in[0,1], p3 = RATE * (1-p2)
#        n           : number of samples    

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from math import log, sqrt, ceil

PATH = 'plots/ternary/'
if not os.path.exists(PATH):
    os.makedirs(PATH)
RATE = 0.5
n       =  100

"""
Study the behavior of sum of n ternary random variables
taking values in {z1,z2,z3} with probability (p1,p2,p3).
"""
npoints = 100 # number of different p2
window = 1 # report moving average

RV = np.array([0, 1/2, 1]) # support of Z = {z1,z2,z3}
a, b = RV[0], RV[2] # range of Z
p2 = np.linspace(0., 1., npoints)
p3 = (1-p2) * RATE
p1 = (1-p2) * (1-RATE)
Probs = np.array([p1,p2,p3]).transpose() # probability distribution
delta = 0.05

""" compute the bounds """
kl_bound = np.zeros(npoints) # kl
EB_bound = np.zeros(npoints) # Empirical Bernstein
UB_bound = np.zeros(npoints) # Unexpected Bernstein
Skl_bound = np.zeros(npoints) # Split-kl


for i in range(npoints):
    # Define the probability distribution
    P = Probs[i]
    Ez = np.dot(P,RV) # true mean
    
    # Sample according to the probability distribution
    Sz = np.random.choice(RV, n, p=P)
    emp_Sz, var_Sz, z2 = Emp_MaV(Sz)
    
    # kl bound, rescale the r.v. to [0,1] to apply kl
    kl_bound[i] = (b-a)*solve_kl_sup((emp_Sz-a)/(b-a), log((n+1)/delta)/n) + a - emp_Sz
    
    # Empirical Bernstein bound
    EB_bound[i] = Emp_Bernstein(emp_Sz, var_Sz, a, b, n, delta) - emp_Sz
    
    # Unexpected Bernstein bound
    UB_bound[i] = Unexp_Bernstein(emp_Sz, z2, b, n, delta)

    # Split-kl
    mu = RV[1] # middle value
    Szp = np.maximum(np.zeros(n), Sz-mu)
    Szm = np.maximum(np.zeros(n), mu-Sz)
    Skl_bound[i] = splitkl(mu, a, b, np.mean(Szp), np.mean(Szm), log(2/delta)/n) - emp_Sz

""" Compute the moving average """
ma_kl = np.array([np.mean(kl_bound[i:i+window]) for i in range(npoints-window+1)])
ma_EB = np.array([np.mean(EB_bound[i:i+window]) for i in range(npoints-window+1)])
ma_UB = np.array([np.mean(UB_bound[i:i+window]) for i in range(npoints-window+1)])
ma_Skl = np.array([np.mean(Skl_bound[i:i+window]) for i in range(npoints-window+1)])

plt.rcParams.update({
    'font.size': 30,
    'text.usetex': True,
    'text.latex.preamble': r'\usepackage{amsfonts}'
})

""" Plot """
def plot():
    fig = plt.figure(figsize=(12,9.6))
    plt.plot(p2[:npoints-window+1], ma_kl, label='kl', linewidth=1.6)
    plt.plot(p2[:npoints-window+1], ma_EB, label='Emp-Bern', linewidth=1.6)
    plt.plot(p2[:npoints-window+1], ma_UB, label='Unexp-Bern', linewidth=1.6)
    plt.plot(p2[:npoints-window+1], ma_Skl, label='Split-kl', linewidth=1.6)
    title = r"$n=$"+str(n)+r", $p_1$="+str(RATE)+r"$(1-p_0)$"
    plt.title(title, fontsize=50)
    plt.ylabel(r"Bound $ - \hat{p}$", fontsize=50, fontweight='bold')
    plt.xlabel(r"$p_{0}$", fontsize=50, fontweight='bold')
    plt.xlim(0,1)
    plt.legend()
    plt.tight_layout()
    filename = "n"+str(n)+"_RATE"+str(RATE)
    path=PATH+filename+'.png'
    plt.savefig(path)
    plt.close()  
plot()